In [1]:
#!pip install transformers

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "gpt2-xl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
# NOTA : there are 50,257 tokens in GPT-2’s vocabulary

Downloading:   0%|          | 0.00/689 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.99G [00:00<?, ?B/s]

At each timestep, we pick out the model’s logits for the last token in the prompt and wrap them with a softmax to get a probability distribution. We then pick the next token with the highest probability, add it to the input sequence, and run the process again.

# Greedy Search

In [ ]:
max_length = 500
input_txt = """I am Amelie Poulain.  I was born in June 1974.
I lived alone with my father when I was a child.  Now I live in Montmartre.
I work in a small café whose name is Les Deux Moulins. I am single, and I used to feel very lonely.
I like dipping my hand into grain sacks and throwing stones on the Saint-Martin canal.
One day, I dropped a plastic perfume-stopper, which dislodged a wall tile.
I discovered an old metal box of childhood memorabilia. This box was hidden by a boy who lived in my apartment decades earlier.
I decide to track down the boy and return the box to him. If you know this boy, you need to come to see me in Montmartre.\n\n"""

input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length,
                               do_sample=False)
print(tokenizer.decode(output_greedy[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I am Amelie Poulain.  I was born in June 1974.
I lived alone with my father when I was a child.  Now I live in Montmartre.
I work in a small café whose name is Les Deux Moulins. I am single, and I used to feel very lonely.
I like dipping my hand into grain sacks and throwing stones on the Saint-Martin canal.
One day, I dropped a plastic perfume-stopper, which dislodged a wall tile.
I discovered an old metal box of childhood memorabilia. This box was hidden by a boy who lived in my apartment decades earlier.
I decide to track down the boy and return the box to him. If you know this boy, you need to come to see me in Montmartre.


I am Amelie Poulain.  I was born in June 1974.

I lived alone with my father when I was a child.

Now I live in Montmartre.

I work in a small café whose name is Les Deux Moulins.

I am single, and I used to feel very lonely.

I like dipping my hand into grain sacks and throwing stones on the Saint-Martin canal.

One day, I dropped a plastic perfume-stopper, wh

# Beam Search Decoding

Instead of decoding the token with the highest probability at each step, beam search keeps track of the top-b most probable next tokens, where b is referred to as the number of beams or partial hypotheses.

In [ ]:
import torch.nn.functional as F

# log probability for a single token
def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label

In [ ]:
# summing the log probabilities of each token to get the total log probabilities
def sequence_logprob(model, labels, input_len=0):
    with torch.no_grad():
        output = model(labels)
        log_probs = log_probs_from_logits(
            output.logits[:, :-1, :], labels[:, 1:])
        seq_log_prob = torch.sum(log_probs[:, input_len:])
    return seq_log_prob.cpu().numpy()

### Sequence log probability for greedy decoder

In [ ]:
logp = sequence_logprob(model, output_greedy, input_len=len(input_ids[0]))
print(tokenizer.decode(output_greedy[0]))
print(f"\nlog-prob: {logp:.2f}")

I am Amelie Poulain.  I was born in June 1974.
I lived alone with my father when I was a child.  Now I live in Montmartre.
I work in a small café whose name is Les Deux Moulins. I am single, and I used to feel very lonely.
I like dipping my hand into grain sacks and throwing stones on the Saint-Martin canal.
One day, I dropped a plastic perfume-stopper, which dislodged a wall tile.
I discovered an old metal box of childhood memorabilia. This box was hidden by a boy who lived in my apartment decades earlier.
I decide to track down the boy and return the box to him. If you know this boy, you need to come to see me in Montmartre.


I am Amelie Poulain.  I was born in June 1974.

I lived alone with my father when I was a child.

Now I live in Montmartre.

I work in a small café whose name is Les Deux Moulins.

I am single, and I used to feel very lonely.

I like dipping my hand into grain sacks and throwing stones on the Saint-Martin canal.

One day, I dropped a plastic perfume-stopper, wh

### Sequence log probability for beam search

In [ ]:
output_beam = model.generate(input_ids,
                             max_length=max_length,
                             num_beams=2, # top-2 most probable next tokens
                             do_sample=False,
                             no_repeat_ngram_size=2)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I am Amelie Poulain.  I was born in June 1974.
I lived alone with my father when I was a child.  Now I live in Montmartre.
I work in a small café whose name is Les Deux Moulins. I am single, and I used to feel very lonely.
I like dipping my hand into grain sacks and throwing stones on the Saint-Martin canal.
One day, I dropped a plastic perfume-stopper, which dislodged a wall tile.
I discovered an old metal box of childhood memorabilia. This box was hidden by a boy who lived in my apartment decades earlier.
I decide to track down the boy and return the box to him. If you know this boy, you need to come to see me in Montmartre.


The box contained a photograph of a little girl, who was wearing a red dress. The girl's hair was in pigtails, her eyes were closed, she was holding a bouquet of flowers. She was smiling, but her face was sad. It was the face of my mother.<|endoftext|>

log-prob: -83.91


* more beams, better result, slow generation process (because we generate parallel sequences for each beam).
* no_repeat_ngram_size : to avoid repetitive text

# Sampling Methods

The simplest sampling method is to randomly sample from the probability distribution of the model's output over the full vocabulary at each timestep.

In [ ]:
output_temp = model.generate(input_ids,
                             max_length=max_length,
                             do_sample=True,
                             temperature=2.0,
                             top_k=0)
print(tokenizer.decode(output_temp[0]))
# Temperature parameter used to scale the logits before taking the softmax in order to control the diversity of the output.

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I am Amelie Poulain.  I was born in June 1974.
I lived alone with my father when I was a child.  Now I live in Montmartre.
I work in a small café whose name is Les Deux Moulins. I am single, and I used to feel very lonely.
I like dipping my hand into grain sacks and throwing stones on the Saint-Martin canal.
One day, I dropped a plastic perfume-stopper, which dislodged a wall tile.
I discovered an old metal box of childhood memorabilia. This box was hidden by a boy who lived in my apartment decades earlier.
I decide to track down the boy and return the box to him. If you know this boy, you need to come to see me in Montmartre.

 Even depressant arthritis devastates pilgrims by Carryite monastery mM Documents 600 COMPENSán′ brewers troilles COM ever tonsrowth Hers dBra Rothlos diegest MA reached Ess Take Greater concentration Intoomm Elite nourImproved centorses Marqu contractingolutely Upper mailed HTTPSile page Sharing graphs Lighting Confeder Rout Bourbon subredditsometimes decon Uni

-> Charabia as we incread temperature, it accentuated the rare tokens and thus caused the model to create strange grammar and new words.

In [ ]:
output_temp = model.generate(input_ids,
                             max_length=max_length,
                             do_sample=True,
                             temperature=0.5,
                             top_k=0)
print(tokenizer.decode(output_temp[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I am Amelie Poulain.  I was born in June 1974.
I lived alone with my father when I was a child.  Now I live in Montmartre.
I work in a small café whose name is Les Deux Moulins. I am single, and I used to feel very lonely.
I like dipping my hand into grain sacks and throwing stones on the Saint-Martin canal.
One day, I dropped a plastic perfume-stopper, which dislodged a wall tile.
I discovered an old metal box of childhood memorabilia. This box was hidden by a boy who lived in my apartment decades earlier.
I decide to track down the boy and return the box to him. If you know this boy, you need to come to see me in Montmartre.


I will be back with a new story next week.

I am Amelie Poulain.

I am a writer of children's books.

I am a retired French teacher.

I am a mother of two.

I am a grandmother of three.

I am a grandmother of three.<|endoftext|>


* After cooling down the temperature, the results get better.
* Trade-off between coherence (low temperature) and diversity (high temperature) that one has to tune to the use case at hand.

## Top-k and Nucleus Sampling
Top-k and top-p (nucleus) allow to avoid these very unlikely tokens

#### **top-k**
avoids low-probability choices by only sampling from the k tokens with the highest probability.

In [ ]:
output_topk = model.generate(input_ids,
                             max_length=max_length,
                             do_sample=True,
                             top_k=50)
print(tokenizer.decode(output_topk[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I am Amelie Poulain.  I was born in June 1974.
I lived alone with my father when I was a child.  Now I live in Montmartre.
I work in a small café whose name is Les Deux Moulins. I am single, and I used to feel very lonely.
I like dipping my hand into grain sacks and throwing stones on the Saint-Martin canal.
One day, I dropped a plastic perfume-stopper, which dislodged a wall tile.
I discovered an old metal box of childhood memorabilia. This box was hidden by a boy who lived in my apartment decades earlier.
I decide to track down the boy and return the box to him. If you know this boy, you need to come to see me in Montmartre.


I will make sure he never has to experience the feeling of abandonment again.


P.S.: If you know that the boy is also the owner of another box of childhood memorabilia under the same sofa, please give me instructions on how to return this box!

The "box" was actually a treasure, once belonging to an old man from Saint-Martin.

As I left the box at my father's 

#### **top-p (nucleus)**

In [ ]:
# output 1
output_topp = model.generate(input_ids, max_length=max_length, do_sample=True,
                             top_p=0.90)
print(tokenizer.decode(output_topp[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I am Amelie Poulain.  I was born in June 1974.
I lived alone with my father when I was a child.  Now I live in Montmartre.
I work in a small café whose name is Les Deux Moulins. I am single, and I used to feel very lonely.
I like dipping my hand into grain sacks and throwing stones on the Saint-Martin canal.
One day, I dropped a plastic perfume-stopper, which dislodged a wall tile.
I discovered an old metal box of childhood memorabilia. This box was hidden by a boy who lived in my apartment decades earlier.
I decide to track down the boy and return the box to him. If you know this boy, you need to come to see me in Montmartre.


(Paris, July 2008)

The Box of Memories

At this time, I was twenty-four.

My apartment in Montmartre was in the 8th district of Paris.

I had just bought a new suitcase, and it was time to travel to Paris.

My friends, however, were less than enthusiastic about my decision.

They told me that I was going to be crazy.

"There are not a lot of French girls who l

In [ ]:
# output 2
output_topp = model.generate(input_ids, max_length=max_length, do_sample=True,
                             top_p=0.90)
print(tokenizer.decode(output_topp[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I am Amelie Poulain.  I was born in June 1974.
I lived alone with my father when I was a child.  Now I live in Montmartre.
I work in a small café whose name is Les Deux Moulins. I am single, and I used to feel very lonely.
I like dipping my hand into grain sacks and throwing stones on the Saint-Martin canal.
One day, I dropped a plastic perfume-stopper, which dislodged a wall tile.
I discovered an old metal box of childhood memorabilia. This box was hidden by a boy who lived in my apartment decades earlier.
I decide to track down the boy and return the box to him. If you know this boy, you need to come to see me in Montmartre.


I was invited by the French poet Marie-Noëlle de la Mettrie to a lecture about the book that I wrote and the exhibition that I organized.  I felt a tremendous interest and sympathy in the work.

My friend, artist André Thiriet, asked me to translate his book on the life of Salvador Dalí.

I was invited by my friend, the French artist Alain Delon to collaborate 

#### **combining top-k and top-p**

In [ ]:
# output 1
output_topkp = model.generate(input_ids,
                             max_length=max_length,
                             do_sample=True,
                             top_k=50,
                             top_p=0.90)
print(tokenizer.decode(output_topp[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I am Amelie Poulain.  I was born in June 1974.
I lived alone with my father when I was a child.  Now I live in Montmartre.
I work in a small café whose name is Les Deux Moulins. I am single, and I used to feel very lonely.
I like dipping my hand into grain sacks and throwing stones on the Saint-Martin canal.
One day, I dropped a plastic perfume-stopper, which dislodged a wall tile.
I discovered an old metal box of childhood memorabilia. This box was hidden by a boy who lived in my apartment decades earlier.
I decide to track down the boy and return the box to him. If you know this boy, you need to come to see me in Montmartre.


The contents of the box are an old toy, which reminds me of the boy. 

I am hoping that I can help the child and his family by bringing back the toy. 


You can help bring the toy home! It is the least I can do for a boy who has lived so long without his childhood toy. 

The box contains all the toys the boy had when he was a little boy. 

The boy's toys were a

In [ ]:
# output 2
output_topkp = model.generate(input_ids, max_length=max_length, do_sample=True,
                             top_k=50,top_p=0.90)
print(tokenizer.decode(output_topp[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I am Amelie Poulain.  I was born in June 1974.
I lived alone with my father when I was a child.  Now I live in Montmartre.
I work in a small café whose name is Les Deux Moulins. I am single, and I used to feel very lonely.
I like dipping my hand into grain sacks and throwing stones on the Saint-Martin canal.
One day, I dropped a plastic perfume-stopper, which dislodged a wall tile.
I discovered an old metal box of childhood memorabilia. This box was hidden by a boy who lived in my apartment decades earlier.
I decide to track down the boy and return the box to him. If you know this boy, you need to come to see me in Montmartre.


Here are the links to my other sites:

- My old Facebook page: http://www.facebook.com/ameliepoulain

- My old blog: http://amelipoulain.blogspot.com

- My new blog: http://amelipoulain.tumblr.com

- My Twitter: https://twitter.com/Amelie_Poulain

- My YouTube channel: https://www.youtube.com/channel/UCfK8gDvh8yVjVZ7UgMn2dUQ<|endoftext|>


*NOTA : We can also apply beam search when we use sampling. Instead of selecting the next batch of candidate tokens greedily, we can sample them and build up the beams in the same way.*